In [1]:
import os
import pandas as pd
import torch
import numpy as np
import cv2

from inference import leish_count_per_image, prepare_model, inference


In [5]:
PATH = 'GABARITO'
PLACA = '013 16 1'
THRESHOLD_LEISHMANIA = 0.15
THRESHOLD_MACROFAGO_CONTAVEL = 0.3
CHECKPOINT_PATH = 'lightning_logs/version_0/checkpoints/epoch=18-step=3096.ckpt'

FOLDER_LIST = os.listdir(os.path.join(PATH, PLACA))

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'


model = prepare_model(CHECKPOINT_PATH, device=DEVICE)


LOADING THE MODEL CHECKPOINT


Some weights of the model checkpoint at nvidia/mit-b3 were not used when initializing SegformerForSemanticSegmentation: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b3 and are newly initialized: ['decode_head.linear_c.3.proj.weight', 'decode_head.classifier.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_c.3.proj.bias', 'decode_head.classifier.weight', 'decode_head.ba

Model loaded to cuda:0!


In [6]:
### INFERENCE ON GABARITO

dataframe_results = pd.DataFrame()
indices_table = {}
writer = pd.ExcelWriter(os.path.join('', PLACA+'.xlsx'), engine='xlsxwriter')
for folder in FOLDER_LIST:
    image_list = os.listdir(os.path.join(PATH, PLACA, folder))

    for image in image_list:
        
        img = cv2.imread(os.path.join(PATH, PLACA, folder, image))
        
        assert isinstance(img, np.ndarray), "Error in loading Image"
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (768, 768))

        img_copy = img.copy()

        print('Realizando inferencia na imagem: ', image)
        outputs = inference(model=model, image=img, device=DEVICE)

        ## THRESHOLD SECTION
        leishmania_segmentation = outputs['leishmania'] > THRESHOLD_LEISHMANIA
        contable_segmentation = outputs['macrofago contavel'] > THRESHOLD_MACROFAGO_CONTAVEL

        # POS PROCESSING TO COUNT LEISHMANIA ONLY WITH CONTABLE MACROPHAGES
        n_leish = leish_count_per_image(contable_segmentation=contable_segmentation,leish_segmentation=leishmania_segmentation)

        

        results = {'Contagem': n_leish, 'Imagem': image} # 'Laminula': folder, 'Placa': PLACA}
        df_conts = pd.DataFrame.from_dict(results)
        dataframe_results = pd.concat([dataframe_results, df_conts])
 


    dataframe_results.to_excel(writer, sheet_name=folder)
    

    quantidade_macrofago_com_leish = (dataframe_results['Contagem'] != 0).sum()
    total_leish = dataframe_results['Contagem'].sum()
    infection_ratio = total_leish/quantidade_macrofago_com_leish
    indices_table[folder] = {'Indice de infecção': infection_ratio, 'Quantidade de leishmania': total_leish,
        'Quantidade de macrofagos com leishmania': quantidade_macrofago_com_leish, 'Total de Macrofagos': len(dataframe_results)
    }
    
    # reset dataframe
    dataframe_results = pd.DataFrame()


dataframe_ratio_infections = pd.DataFrame.from_dict(indices_table)
dataframe_ratio_infections.to_excel(writer, sheet_name='FINAL')

writer.close()

    

Realizando inferencia na imagem:  20.jpg
Realizando inferencia na imagem:  21.jpg
Realizando inferencia na imagem:  22.jpg
Realizando inferencia na imagem:  23.jpg
Realizando inferencia na imagem:  24.jpg
Realizando inferencia na imagem:  25.jpg
Realizando inferencia na imagem:  26.jpg
Realizando inferencia na imagem:  27.jpg
Realizando inferencia na imagem:  28.jpg
Realizando inferencia na imagem:  29.jpg
Realizando inferencia na imagem:  30.jpg
Realizando inferencia na imagem:  31.jpg
Realizando inferencia na imagem:  32.jpg
Realizando inferencia na imagem:  33.jpg
Realizando inferencia na imagem:  34.jpg
Realizando inferencia na imagem:  35.jpg
Realizando inferencia na imagem:  36.jpg
Realizando inferencia na imagem:  37.jpg
Realizando inferencia na imagem:  38.jpg
Realizando inferencia na imagem:  39.jpg
Realizando inferencia na imagem:  201.jpg
Realizando inferencia na imagem:  202.jpg
Realizando inferencia na imagem:  203.jpg
Realizando inferencia na imagem:  204.jpg
Realizando i